In [8]:
import pandas as pd
import os
import numpy as np
# from tqdm.notebook import tqdm_notebook
import random as rn

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model


rn.seed(42)
np.random.seed(42)

tf.random.set_seed(42)

In [9]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# !pip install tqdm

In [10]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5886132950047101990
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7787773952
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11014889843603004407
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [33]:
import pandas as pd
# df =pd.read_csv("0_jy_original_2022.csv")
# df = pd.read_csv("0_jy_interpol.csv")
# nwp_cols = "lead_hr,NDNSW,SWDIR,SWDIF,TDSWS,NDNLW,OULWT,DLWS,NCPCP,SNOL,LSPRATE,LSSRATE,HPBLA,HFSOIL,LLRIB,UGRD,VGRD,HFSFC,XBLWS,YBLWS,LHTFL,TMP,TMIN,TMAX,SPFH,TOMFS,RH,VIS,FOGFR,DPT,PVIS5,15TL,15QT,VISIP,SHFLT-1,SHFLT-2,SHFLT-3,SHFLT-4,SHFLT-5,SHFLT-6,SHFLT-7,SHFLT-8,SHFLT-9,HTBM,SCST,DSCNC,DSCOC,NETTL-1,NETTL-2,NETTL-3,NETTL-4,NETTL-5,NETTL-6,NETTL-7,NETTL-8,NETTL-9,STOT-1,STOT-2,STOT-3,STOT-4,STOT-5,STOT-6,STOT-7,STOT-8,STOT-9,HCNP-1,HCNP-2,HCNP-3,HCNP-4,HCNP-5,WCNP-1,WCNP-2,WCNP-3,WCNP-4,WCNP-5,T15T-1,T15T-2,T15T-3,T15T-4,T15T-5,T15T-6,T15T-7,T15T-8,T15T-9,SH15-1,SH15-2,SH15-3,SH15-4,SH15-5,SH15-6,SH15-7,SH15-8,SH15-9,HDLB,STCP-1,STCP-2,STCP-3,STCP-4,STCP-5,MAXGUST,CBLT,SNOM,SMCL-1,SMCL-2,SMCL-3,SMCL-4,TSOIL-1,TSOIL-2,TSOIL-3,TSOIL-4,ROFR,SROFR,VLCDC,LCDC,MCDC,HCDC,CB25,CB45,TCAR,TCAM,CFB10,LCB,WBFLH,TCTH,50MU-min,50MV-min,50MU-max,50MV-max,PRMSL,SNOAL,TMP-SFC,HPBL,SFCR,LAND,FRICE,DIST,PRES"
df = pd.read_csv("clean_dataset/s1_case1.csv")
nwp_cols="lead_hr,NDNSW,SWDIR,SWDIF,TDSWS,NDNLW,OULWT,DLWS,HPBLA,HFSOIL,LLRIB,UGRD,VGRD,HFSFC,XBLWS,YBLWS,LHTFL,TMP,TMIN,TMAX,SPFH,TOMFS,RH,VIS,DPT,15TL,15QT,VISIP,SHFLT-1,SHFLT-2,SHFLT-3,SHFLT-4,SHFLT-5,SHFLT-6,SHFLT-7,SHFLT-8,HTBM,NETTL-1,NETTL-2,NETTL-3,NETTL-4,NETTL-5,NETTL-6,NETTL-7,NETTL-8,STOT-1,STOT-2,STOT-3,STOT-4,STOT-5,STOT-6,STOT-7,STOT-8,STOT-9,T15T-1,T15T-2,T15T-3,T15T-4,T15T-8,SH15-1,SH15-2,SH15-3,SH15-4,SH15-8,STCP-1,STCP-2,STCP-3,STCP-4,MAXGUST,CBLT,SNOM,SMCL-1,SMCL-2,SMCL-3,SMCL-4,TSOIL-1,TSOIL-2,TSOIL-3,TSOIL-4,ROFR,SROFR,VLCDC,LCDC,MCDC,HCDC,TCAR,TCAM,LCB,TCTH,50MU-min,50MV-min,50MU-max,50MV-max,PRMSL,SNOAL,TMP-SFC,HPBL,PRES"
nwp_cols = nwp_cols.split(',')
output_col = ["OT"]
FCST_cols =['date']
# CRTN_cols = ['CRTN_TM']
all_cols = FCST_cols+nwp_cols+output_col
tmp_df= df[all_cols]

tmp_df.date = pd.to_datetime(tmp_df.date)

# case1 
# case11_df= tmp_df.copy()
# case11_df=case11_df[case11_df['lead_hr']!=0]
# # case11_df=case11_df.drop(columns=['CRTN_TM'])
# case11_df.reset_index(inplace=True)
# case11_df=case11_df.drop(columns=['index'])


# case2
# tmp_df['day']=tmp_df.date.dt.day
# case11_df= tmp_df.copy()
# case11_df=case11_df[case11_df['lead_hr']!=0]
# train_df = case11_df[case11_df['day']<=25]
# test_df = case11_df[case11_df['day']>25]
# case11_df = pd.concat([train_df,test_df])
# case11_df=case11_df.drop(columns=['day'])
# case11_df.reset_index(inplace=True)
# case11_df=case11_df.drop(columns=['index'])

# case3 
case11_df= tmp_df.copy()
case11_df=case11_df[case11_df['lead_hr']!=0]
case11_df['month'] = case11_df.date.dt.month
case11_df['day']=case11_df.date.dt.day
test_df=case11_df[(case11_df['month'].isin([5,8,11,2]))&(case11_df['day'] > 12)]
train_df=case11_df.drop(index=list(test_df.index))
case11_df = pd.concat([train_df,test_df])
case11_df=case11_df.drop(columns=['day','month'])
case11_df.reset_index(inplace=True)
case11_df=case11_df.drop(columns=['index'])


In [34]:
case11_df=case11_df.set_index("date")

In [35]:

from sklearn.preprocessing import StandardScaler

# sc = StandardScaler() 
# x = case11_df.drop(['OT'], axis=1).values # 독립변인들의 value값만 추출
# y = case11_df['OT'].values # 종속변인 추출
# x_std = sc.fit_transform(x) # standardizing the data
# x_std_df= pd.DataFrame(x_std, columns=case11_df.columns)

# case11_df=case11_df.reset_index()
# case11_df=case11_df[['date','OT']]
# case11_df=pd.merge(case11_df,x_std_df,how='inner',left_index=True,right_index=True)
# case11_df.head()

sc = StandardScaler()
x = case11_df.values
x_std = sc.fit_transform(x)
x_std_df = pd.DataFrame(x_std, columns = case11_df.columns)

case11_df = case11_df.reset_index()
case11_df=case11_df.reset_index()
case11_df=case11_df[['date']]
case11_df=pd.merge(case11_df,x_std_df,how='inner',left_index=True,right_index=True)


In [36]:
all_len = len(case11_df)
case11_df.date = pd.to_datetime(case11_df.date)
# data = data[data.OT > 0]

training_original_df = case11_df[:int(all_len*0.8)].reset_index(drop=True)
test_original_df = case11_df[int(all_len*0.8):].reset_index(drop=True)
test_original_df=test_original_df[(test_original_df['date'].dt.hour>=7) &(test_original_df['date'].dt.hour<20)]
test_original_df=test_original_df.reset_index(drop=True)

training_df = training_original_df[FCST_cols+nwp_cols+output_col]

train_X = training_df[nwp_cols].values
train_y = training_df['OT'].values

test_X = test_original_df[nwp_cols].values
test_y = test_original_df['OT'].values
test_df = test_original_df


In [37]:
train_X = train_X.reshape(-1, len(nwp_cols), 1)
test_X = test_X.reshape(-1, len(nwp_cols), 1)

train_y = train_y.reshape(-1, 1)
test_y = test_y.reshape(-1, 1)

In [38]:
initializer = tf.keras.initializers.GlorotUniform(seed=42)

def jy_lstm_model(input_cols):
    ## nwp model
    mim_input1 = layers.Input(shape=(len(input_cols),1))
    mim_out1 = layers.LSTM(32, return_sequences=True)(mim_input1)
    mim_out1 = layers.BatchNormalization()(mim_out1)
    mim_out1 = layers.LSTM(16, return_sequences=True)(mim_out1)
    mim_out1 = layers.BatchNormalization()(mim_out1)
    mim_out1 = layers.LSTM(8, return_sequences=True)(mim_out1)
    mim_out1 = layers.BatchNormalization()(mim_out1)
    mim_out1 = layers.Flatten()(mim_out1)

    concatenated = layers.Dense(256, activation='relu')(mim_out1)
    # concatenated = layers.Dropout(0.2)(concatenated)
    concatenated = layers.BatchNormalization()(concatenated)
    concatenated = layers.Dense(64, activation='relu')(concatenated)
    concatenated = layers.BatchNormalization()(concatenated)
    concatenated = layers.Dense(8, activation='relu')(concatenated)
    concatenated = layers.BatchNormalization()(concatenated)
    concat_out = layers.Dense(1, activation='linear')(concatenated)

    concat_model = Model(mim_input1, concat_out)

    return concat_model

In [39]:
## validation MAPE, RMSE 기준으로 EarlyStopping
### input : patience
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, patience=0):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best_v_mape = np.Inf
        self.best_v_rmse = np.Inf

    def on_epoch_end(self, epoch, logs=None): 
        v_mape=logs.get('val_mean_absolute_percentage_error')
        v_rmse=logs.get('val_mean_squared_error')
        # v_rmse = logs.get('val_myLoss')

        if np.less(v_mape, self.best_v_mape) and np.less(v_rmse, self.best_v_rmse):
            self.best_v_mape = v_mape
            self.best_v_rmse = v_rmse
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)
                
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))


In [40]:
## model define
from datetime import datetime
initializer = tf.keras.initializers.GlorotUniform(seed=42)
model = jy_lstm_model(nwp_cols)
model.summary()
plot_model(model, to_file='lstm_graph.png')
model.compile(optimizer='adam', loss="mse", metrics=[tf.keras.losses.MeanAbsolutePercentageError(),tf.keras.metrics.MeanSquaredError()])
start_dt = datetime.now()
# model fit
model.fit(train_X, train_y, 
          epochs=1000, batch_size=256,
          callbacks=[CustomEarlyStopping(patience=10)],
          validation_data=(test_X, test_y),
          shuffle=True, verbose=2)	# because of time-series


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 98, 1)]           0         
                                                                 
 lstm_12 (LSTM)              (None, 98, 32)            4352      
                                                                 
 batch_normalization_24 (Bat  (None, 98, 32)           128       
 chNormalization)                                                
                                                                 
 lstm_13 (LSTM)              (None, 98, 16)            3136      
                                                                 
 batch_normalization_25 (Bat  (None, 98, 16)           64        
 chNormalization)                                                
                                                                 
 lstm_14 (LSTM)              (None, 98, 8)             800 

In [41]:
# concat_model.save("convlstm_model_customloss")
model.save("model/jy_lstm_model_case1_2.h5")
model = keras.models.load_model("model/jy_lstm_model_case1_2.h5", compile=False)
test_start_dt = datetime.now()
prediction = model.predict(test_X)


prediction_df = pd.DataFrame(prediction, columns=['OT_pred'])
# prediction_df[prediction_df['adjust_pred']<0]=0
# test_df=test_df.reset_index()
test_df["OT_pred"] = prediction_df


from sklearn.metrics import mean_absolute_error,mean_squared_error
mae = mean_absolute_error(test_df['OT'], test_df['OT_pred'])
mse = mean_squared_error(test_df['OT'], test_df['OT_pred'])

print(mse)
print(mae)
print((test_start_dt- start_dt).total_seconds())
print((datetime.now()-test_start_dt).total_seconds())

233/233 [==============================] - 3s 11ms/step
0.20701295408205295
0.3188444599323698
85.044384
3.321943


In [50]:
# data.CRTN_TM = pd.to_datetime(data.CRTN_TM)
data.date = pd.to_datetime(data.date)
# data = data[data.OT > 0]

training_original_df = data[data.date < '2022-10-01 00:00:00'].reset_index(drop=True)
test_original_df = data[data.date >= '2022-10-01 00:00:00'].reset_index(drop=True)

training_df = training_original_df[all_cols]

train_X = training_df[nwp_cols].values
train_y = training_df[['OT']].values

test_X = test_original_df[nwp_cols].values
test_y = test_original_df[['OT']].values
test_df = test_original_df

C:\Users\HONGSJ\AppData\Local\Temp\ipykernel_60404\2734462485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.date = pd.to_datetime(data.date)


In [51]:
from sklearn.preprocessing import MinMaxScaler

sc_x1 = MinMaxScaler()
train_X = sc_x1.fit_transform(train_X)
test_X = sc_x1.transform(test_X)

sc_y = MinMaxScaler()
train_y = sc_y.fit_transform(train_y)
test_y = sc_y.transform(test_y)

In [52]:
train_X = train_X.reshape(-1, len(nwp_cols), 1)
test_X = test_X.reshape(-1, len(nwp_cols), 1)

train_y = train_y.reshape(-1, 1)
test_y = test_y.reshape(-1, 1)

In [53]:
train_X.shape

(45602, 137, 1)

In [54]:
test_X.shape

(17523, 137, 1)

In [55]:
train_y.shape

(45602, 1)

In [56]:
test_y.shape

(17523, 1)

In [57]:
initializer = tf.keras.initializers.GlorotUniform(seed=42)

def jy_lstm_model(input_cols):
    ## nwp model
    mim_input1 = layers.Input(shape=(len(input_cols),1))
    mim_out1 = layers.LSTM(32, return_sequences=True)(mim_input1)
    mim_out1 = layers.BatchNormalization()(mim_out1)
    mim_out1 = layers.LSTM(16, return_sequences=True)(mim_out1)
    mim_out1 = layers.BatchNormalization()(mim_out1)
    mim_out1 = layers.LSTM(8, return_sequences=True)(mim_out1)
    mim_out1 = layers.BatchNormalization()(mim_out1)
    mim_out1 = layers.Flatten()(mim_out1)

    concatenated = layers.Dense(256, activation='relu')(mim_out1)
    # concatenated = layers.Dropout(0.2)(concatenated)
    concatenated = layers.BatchNormalization()(concatenated)
    concatenated = layers.Dense(64, activation='relu')(concatenated)
    concatenated = layers.BatchNormalization()(concatenated)
    concatenated = layers.Dense(8, activation='relu')(concatenated)
    concatenated = layers.BatchNormalization()(concatenated)
    concat_out = layers.Dense(1, activation='linear')(concatenated)

    concat_model = Model(mim_input1, concat_out)

    return concat_model

In [58]:
## validation MAPE, RMSE 기준으로 EarlyStopping
### input : patience
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, patience=0):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best_v_mape = np.Inf
        self.best_v_rmse = np.Inf

    def on_epoch_end(self, epoch, logs=None): 
        v_mape=logs.get('val_mean_absolute_percentage_error')
        v_rmse=logs.get('val_mean_squared_error')
        # v_rmse = logs.get('val_myLoss')

        if np.less(v_mape, self.best_v_mape) and np.less(v_rmse, self.best_v_rmse):
            self.best_v_mape = v_mape
            self.best_v_rmse = v_rmse
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)
                
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))


In [22]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: D:\study\tensorflow\Scripts\python.exe -m pip install --upgrade pip


In [59]:
## model define
initializer = tf.keras.initializers.GlorotUniform(seed=42)
model = jy_lstm_model(nwp_cols)
model.summary()
plot_model(model, to_file='lstm_graph.png')
model.compile(optimizer='adam', loss="mse", metrics=[tf.keras.losses.MeanAbsolutePercentageError(),tf.keras.metrics.MeanSquaredError()])

# model fit
model.fit(train_X, train_y, 
          epochs=1000, batch_size=256,
          callbacks=[CustomEarlyStopping(patience=10)],
          validation_data=(test_X, test_y),
          shuffle=True, verbose=2)	# because of time-series


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 137, 1)]          0         
                                                                 
 lstm_9 (LSTM)               (None, 137, 32)           4352      
                                                                 
 batch_normalization_18 (Bat  (None, 137, 32)          128       
 chNormalization)                                                
                                                                 
 lstm_10 (LSTM)              (None, 137, 16)           3136      
                                                                 
 batch_normalization_19 (Bat  (None, 137, 16)          64        
 chNormalization)                                                
                                                                 
 lstm_11 (LSTM)              (None, 137, 8)            800 

In [60]:
# concat_model.save("convlstm_model_customloss")
model.save("model/jy_lstm_model_case1_2.h5")
model = keras.models.load_model("model/jy_lstm_model_case1_2.h5", compile=False)

prediction = model.predict(test_X)

real_predictions = sc_y.inverse_transform(prediction)
# real_predictions = prediction
real_predictions = real_predictions.reshape(real_predictions.shape[0]*real_predictions.shape[1])


548/548 [==============================] - 8s 13ms/step


In [79]:
test_df = test_original_df
prediction_df = pd.DataFrame(real_predictions, columns=['adjust_pred'])
# prediction_df[prediction_df['adjust_pred']<0]=0

test_df["OT_pred"] = prediction_df
test_df= test_df[test_df['OT']>0]

# print(test_df)
from sklearn.metrics import mean_absolute_error,mean_squared_error
mae=mean_absolute_error(test_df['OT'], test_df['OT_pred'])
mse = mean_squared_error(test_df['OT'], test_df['OT_pred'])
print(mse,mae)
test_df['n_mape'] = abs(test_df["OT"] - test_df["OT_pred"]) /2184 *100
print(round(np.mean(test_df['n_mape']),2))

160524.02089659084 313.7804043357511
14.37


C:\Users\HONGSJ\AppData\Local\Temp\ipykernel_60404\2819021567.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['n_mape'] = abs(test_df["OT"] - test_df["OT_pred"]) /2184 *100


In [80]:
test_df

,date,lead_hr,NDNSW,SWDIR,SWDIF,TDSWS,NDNLW,OULWT,DLWS,NCPCP,...,HPBL,SFCR,LAND,FRICE,DIST,PRES,OT,CRTN_TM,OT_pred,n_mape
49,2022-10-01 06:00:00,21,0.000000,0.000000,0.000000,0.000000,-88.156021,289.040649,298.502487,0.0,...,89.112728,0.500002,1,9999,393.8125,97151.31250,118.465833,2022-09-30 9:00,228.500687,5.038226
50,2022-10-01 06:00:00,45,0.000000,0.000000,0.000000,0.000000,-87.287666,288.359650,302.056839,0.0,...,83.923545,0.500002,1,9999,393.8125,97162.28125,118.465833,2022-09-29 9:00,161.545593,1.972516
51,2022-10-01 06:00:00,3,0.000000,0.000000,0.000000,0.000000,-89.709282,285.835861,284.607010,0.0,...,83.947447,0.500002,1,9999,393.8125,97131.33594,118.465833,2022-10-01 3:00,68.721741,2.277660
52,2022-10-01 06:00:00,33,0.000000,0.000000,0.000000,0.000000,-89.448166,288.545624,298.564529,0.0,...,89.113375,0.500002,1,9999,393.8125,97148.36719,118.465833,2022-09-29 21:00,229.032272,5.062566
53,2022-10-01 06:00:00,9,0.000000,0.000000,0.000000,0.000000,-94.724327,290.485153,287.944519,0.0,...,89.101119,0.500002,1,9999,393.8125,97096.20313,118.465833,2022-09-30 21:00,97.481735,0.960810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17469,2022-12-31 17:00:00,8,72.356906,35.335938,46.778117,82.077874,-78.455750,242.157303,227.652084,0.0,...,115.577175,0.500002,1,9999,393.8125,97934.85156,5.750278,2022-12-31 9:00,0.000000,0.263291
17470,2022-12-31 17:00:00,32,47.380344,8.406250,45.611020,53.635316,-35.829834,233.529205,276.171921,0.0,...,778.849898,0.500002,1,9999,393.8125,97929.92188,5.750278,2022-12-30 9:00,280.230133,12.567759
17471,2022-12-31 17:00:00,2,76.208469,40.691406,46.489055,86.241797,-87.699585,244.750015,222.613052,0.0,...,207.476852,0.500002,1,9999,393.8125,97962.66406,5.750278,2022-12-31 15:00,0.000000,0.263291
17472,2022-12-31 17:00:00,38,64.028781,23.507812,49.189145,72.463441,-63.962235,238.142166,247.639908,0.0,...,667.351851,0.500002,1,9999,393.8125,97913.53125,5.750278,2022-12-30 3:00,179.239960,7.943667
